# Agentic AI Workshop

# Introducing: The Price Is Right

We are going to build a multi-agent framework that will:
1. Pull RSS feeds from the web and interpret them, looking for promising deals (multi-shot prompting, structured outputs)
2. Use RAG and a Frontier Model to estimate the price of a product, using a knowledge base
3. Use a QLoRA fine tuned LLM to estimate the price
4. Send a push notification when it finds a deal that's worth significantly less than it estimates
5. A planning agent will orchestrate this

If you're having any set up problems, please see the [troubleshooting](../troubleshooting.ipynb) notebook in the parent directory.  
For more details on using Jupyter Lab, please see the [Guide To Jupyter](../Guide%20to%20Jupyter.ipynb) notebook.  
If you'd like to brush up your Python skills, there's an [Intermediate Python](../Intermediate%20Python.ipynb) notebook.


## First: Let's look at the Agent code for interpreting RSS feeds, including figuring out the pricing, using multi-shot prompting

## Remember: just get some intuition for this, and run it yourself later..

In [1]:
# Some imports

from dotenv import load_dotenv
from agents.deals import ScrapedDeal
from openai import OpenAI
import logging

In [2]:
# Set up our env variables as before

load_dotenv()
openai = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# I wrote this code to fetch RSS feeds

scraped_deals = ScrapedDeal.fetch(show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:18<00:00,  6.30s/it]


In [5]:
print(scraped_deals[2].describe())

Title: Soundcore by Anker Life A3i Noise Cancelling Earbuds for $35 + free shipping
Details: Coupon code "SAEN2A79" takes $15 off. Buy Now at Newegg
Features: 36 hours playtime 22 custom EQ transparency mode Model: A39920F1
URL: https://www.dealnews.com/products/Soundcore-by-Anker/Soundcore-by-Anker-Life-A3-i-Noise-Cancelling-Earbuds/395990.html?iref=rss-c142


## We are going to ask GPT-4o-mini to summarize deals and identify their price

In [6]:
SYSTEM_PROMPT = """You identify and summarize the 4 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 4 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
{
    "deals": [
        {
            "product_description": "Your clearly expressed summary of the product in 3-4 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
            "price": 99.99,
            "url": "the url as provided"
        },
        ...
    ]
}"""

In [7]:
USER_PROMPT_PREFIX = """Respond with the most promising 4 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 4 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
Deals:
    
"""

USER_PROMPT_SUFFIX = """\n\nStrictly respond in JSON and include exactly 4 deals, no more.
Your response must follow this format:

{
    "deals": [
        {
            "product_description": "Your summary of the product in 3-4 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
            "price": 99.99,
            "url": "the url as provided"
        },
        {
            "product_description": "Your summary of the product in 3-4 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
            "price": 210.30,
            "url": "the url as provided"
        },
        ...
]}
"""

In [8]:
# this makes a suitable user prompt given scraped deals

def make_user_prompt(scraped):
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([scrape.describe() for scrape in scraped])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [9]:
# Let's create a user prompt for the deals we just scraped, and look at how it begins

user_prompt = make_user_prompt(scraped_deals)
print(user_prompt[:2000])

Respond with the most promising 4 deals from this list, selecting those which have the most detailed, high quality product description and a clear price that is greater than 0.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 4 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 
    
Deals:
    
Title: MountFTV Full Motion TV Wall Mount for 37" to 82" TVs for $22 + free shipping w/ $35
Details: Clip the $11 coupon on the product page to get this deal. (You must be logged in to your Walmart account to see the clippable coupon. It's free to join.) That's $4 under our December mention and the best price we've seen. Buy Now at Walmart
Fe

## Calling OpenAI using their Python client

Recall the format of OpenAI calls:

When you call OpenAI, you pass in your conversation as a list of python dictionaries:

```
    [
        {"role": "system", "content": "system_message_goes_here"},
        {"role": "user", "content": "user_prompt_goes_here"}
    ]
```

In [10]:
# Create the user prompt that reflects the deals we have scraped

user_prompt = make_user_prompt(scraped_deals)

# Call OpenAI using their Python client. Request a response in JSON and stream it back.

stream = openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
  ],
    max_tokens=1000,
    stream=True,
    response_format={"type": "json_object"}
)

response = ""
for chunk in stream:
    chunk_text = chunk.choices[0].delta.content or ''
    response += chunk_text
    print(chunk_text, end="", flush=True)

{
    "deals": [
        {
            "product_description": "The MountFTV Full Motion TV Wall Mount is designed to accommodate TVs ranging from 37 to 82 inches and can hold weights up to 100 lbs. It features a full motion range that allows for a swivel of 45 degrees and a tilt capability of down 15 degrees or up 5 degrees, ensuring you can achieve the perfect viewing angle. The mount extends 15.4 inches from the wall but can also fold down to just 3.2 inches, making it versatile for various setups.",
            "price": 22.00,
            "url": "https://www.dealnews.com/products/Mount-FTV-Full-Motion-TV-Wall-Mount-for-37-to-82-TVs/485585.html?iref=rss-c142"
        },
        {
            "product_description": "The Klipsch Reference 5.2 Home Theater System delivers an immersive audio experience perfect for movie nights and music enthusiasts. It includes two R-625FA Floorstanding Speakers, two R-12SW Subwoofers, one R-52C Center Speaker, and two R-41M Bookshelf Speakers, offering 

In [ ]:
import json
json.loads(response)

# Putting this into an "Agent"

I've packaged this code into a class called `ScannerAgent`

There are various Agent frameworks that add an abstraction layer, but in our case it's easy just to write the code directly.

In [11]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [12]:
from agents.scanner_agent import ScannerAgent

scanner = ScannerAgent()
scanner.scan()

INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 15 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Output
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:[Scanner Agent] Scanner Agent received 5 selected deals with price>0 from OpenAI


DealSelection(deals=[Deal(product_description='The MountFTV Full Motion TV Wall Mount is designed for TVs ranging from 37" to 82" and can hold up to 100 lbs. It features an extension capability of up to 15.4 inches and can be folded to just 3.2 inches from the wall. The mount offers a 45-degree swivel and tilting options of down 15 degrees and up 5 degrees for optimal viewing angles.', price=22.0, url='https://www.dealnews.com/products/Mount-FTV-Full-Motion-TV-Wall-Mount-for-37-to-82-TVs/485585.html?iref=rss-c142'), Deal(product_description='The SVS SB-4000 Subwoofer delivers powerful and deep bassperformance, making it an excellent addition to any home theater system. With advanced DSP processing and a high-excursion 13-inch driver, it can handle the most impactful soundtracks. Ideal for audiophiles looking for robust sound to enhance their multimedia experience.', price=1399.0, url='https://www.dealnews.com/SVS-Winter-Deal-Days-Up-to-30-off/21713906.html?iref=rss-c142'), Deal(product